# Banglore Home Prices Prediction

### Dataset Source: https://www.kaggle.com/amitabhajoy/bengaluru-house-price-data

### Import Libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20, 10)

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# import dataset

df = pd.read_csv("dataset/Bengaluru_House_Data.csv")
df.head()

In [ ]:
#check the shape of dataset
df.shape

- Dataset contains 9 columns and 13320 rows.

In [ ]:
df.groupby('area_type')['area_type'].agg('count')

In [ ]:
# drop irrelevant columns

df1 = df.drop(['area_type','society','balcony','availability'],axis='columns')
df1.head()

### Data Cleaning

In [ ]:
# check the Na values in columns

df1.isnull().sum()

In [ ]:
# drop na values beacuse dataset already contains huge number of values

df2 = df1.dropna()
df2.isnull().sum()

In [ ]:
# chech the shape of dataset after droping Na values

df2.shape

In [ ]:
df2['size'].unique()

In [ ]:
df2['bhk'] = df2['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df2.head()

In [ ]:
df2['bhk'].unique()

In [ ]:
df2[df2.bhk>20]

In [ ]:
df2.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df2[~df2['total_sqft'].apply(is_float)].head(10)

In [ ]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) ==  2:
        return(float(tokens[0])+float(tokens[1])) /2
    try:
        return float(x)
    except:
        return None
    

In [ ]:
df3 = df2.copy()
df3['total_sqft'] = df3['total_sqft'].apply(convert_sqft_to_num)
df3.head()

## Feature Engineering (Dimenstionality Reduction techniques)

In [ ]:
df4 = df3.copy()
df4['price_per_sqft'] = df4['price']*100000/df4['total_sqft']
df4.head()

In [ ]:
len(df4.location.unique())

In [ ]:
df4.location =  df4.location.apply(lambda x: x.strip())
location_stats = df4.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
len(df4.location.unique())

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]

In [ ]:
df4.location = df4.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df4.location.unique())

In [ ]:
df4.head(10)

## Outlier Detection and Removal

In [ ]:
df4[df4.total_sqft/df4.bhk<300].head()

In [ ]:
df4.shape

In [ ]:
df5 = df4[~(df4.total_sqft/df4.bhk<300)]
df5.shape

In [ ]:
df5.price_per_sqft.describe()

In [ ]:
df5.head()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index=True)
    return df_out

df6 = remove_pps_outliers(df5)
df6.shape

In [ ]:
def plot_scatter_chart(df, location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft, bhk2.price, color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft, bhk3.price, marker='+', color='green',label='3 BHK',s=50)
    plt.xlabel('Total Square Feet Area')
    plt.ylabel('Price')
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df6, "Hebbal")

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis='index')

df7 = remove_bhk_outliers(df6)
df7.shape

In [ ]:
plot_scatter_chart(df7, "Hebbal")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20,10)
plt.hist(df7.price_per_sqft,rwidth=0.8)
plt.xlabel('Price Per Square Feet')
plt.ylabel('Count')

In [ ]:
df7.bath.unique()

In [ ]:
df7[df7.bath>10]

In [ ]:
plt.hist(df7.bath, rwidth=0.8)
plt.xlabel('Number of bathrooms')
plt.ylabel('Count')

In [ ]:
df7[df7.bath>df7.bhk+2]

In [ ]:
df8 = df7[df7.bath<df7.bhk+2]
df8.shape

In [ ]:
df8.head()

In [ ]:
df9 = df8.drop(['size','price_per_sqft'],axis='columns')
df9.head(10)

## Model Building

In [ ]:
dummies = pd.get_dummies(df9.location)
dummies.head(3)

In [ ]:
df10 = pd.concat([df9,dummies.drop('other',axis='columns')],axis='columns')
df10.head()

In [ ]:
df11 = df10.drop('location', axis='columns')
df11.head(3)

In [ ]:
df11.shape

In [ ]:
#X split
X = df11.drop('price', axis='columns')
X.head()

In [ ]:
#target variable
y = df11.price
y.head()

In [ ]:
#data distribution using train_test_split method
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train,y_train)
lr_model.score(X_test,y_test)

In [ ]:
#By applying cross-validation method

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(LinearRegression(), X,y,cv=cv)

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression':{
            'model':LinearRegression(),
            'params':{
                'normalize':[True,False]
            }
        },
        'lasso':{
            'model':Lasso(),
            'params':{
                'alpha':[1,2],
                'selection':['random','cyclic']
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion':['mse','friedman_mse'],
                'splitter':['best','random']
            }
        }
    }
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })
    return pd.DataFrame(scores, columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)